In [4]:
from IPython import get_ipython
!pip install -qq GPy scikit-learn
import pandas as pd
import numpy as np
import sys
import os
from GPy.models import GPRegression
from GPy.kern import RBF
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from scipy.stats import mode
from sklearn.metrics import mean_squared_error
from IPython.display import clear_output
from time import time
from glob import glob
import multiprocessing as mp

## Generate script

In [5]:
%%writefile gp_rbf.py
import pandas as pd
import numpy as np
from GPy.models import GPRegression
from GPy.kern import RBF
import os
import sys
np.random.seed(0)
f_name = sys.argv[1]
recompute = int(sys.argv[2])
compute=True
base_data_path = '../data/common/'
base_result_path = '../raw_results/GP_RBF/'

if os.path.exists(f_name):
    compute = True if recompute else False
    
if compute:
    sub_data = pd.read_pickle(base_data_path+f_name)
    kern = RBF(input_dim=2, active_dims=[0,1], ARD=True)
    model = GPRegression(X = sub_data['trn_val_X'], 
                         Y = sub_data['trn_val_Y'],
                         kernel = kern)
    Yscaler = sub_data['Yscaler']
#     model.kern.lengthscale.constrain_bounded(0.1,10)
    model.optimize_restarts(5, robust=True, verbose=0)
    pred_y = Yscaler.inverse_transform(model.predict(sub_data['test_X'])[0])
    rmse = np.sqrt(np.mean(np.square(sub_data['test_Y'].squeeze() - pred_y.squeeze())))
#     print(rmse, pred_y, model.kern.lengthscale)
    pd.to_pickle({'rmse':rmse, 'test_y':sub_data['test_Y'].squeeze(),
                  'pred_y':pred_y, 'lengthscales':model.kern.lengthscale.tolist(),'model':model}, 
                 base_result_path+f_name)

Overwriting gp_rbf.py


### Compute

In [10]:
i = 0
jobs = []
main_command = 'python gp_rbf.py'

def do_the_job(command):
    os.system(command)

for file in glob('../data/common/*'):
    clear_output(wait=True)
    print(i)
    jobs.append(main_command+' '+file.split('/')[-1]+' '+'1')
    i +=1
print('Appended')

if not os.path.exists('../raw_results/GP_RBF/'):
    os.makedirs('../raw_results/GP_RBF/')

workers = mp.Pool(mp.cpu_count()-1)
init = time()
workers.map(do_the_job, jobs, chunksize=840//(mp.cpu_count()-1))
end = time()-init
assert len(glob('../raw_results/GP_RBF/*')) == 840
os.system('python -V > python_version.txt')
os.system('lscpu > cpu_info.txt')
with open(main_command+'_time.txt', 'w') as f:
    print('time in seconds:',end,file=f)
print('Job finished in',end,'seconds')

839
Appended
Job finished in 89.70398044586182 seconds
